[View in Colaboratory](https://colab.research.google.com/github/shin-sforzando/ctr-forecaster/blob/master/CNN.ipynb)

In [1]:
!pip install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master --upgrade
!jupyter contrib nbextension install --user
!jupyter nbextension enable hinterland/hinterland

     - 26.1MB 65.0MB/s
    100% |████████████████████████████████| 860kB 6.6MB/s 
    100% |████████████████████████████████| 471kB 9.3MB/s 
    100% |████████████████████████████████| 5.8MB 5.5MB/s 
  Running setup.py bdist_wheel for jupyter-contrib-nbextensions ... - \ | / - \ | / - \ | / done
  Stored in directory: /tmp/pip-ephem-wheel-cache-whd3c9ta/wheels/22/4a/9f/df59e985684a10ea0e025300581870b5b3a300ee3525f0eef5
  Running setup.py bdist_wheel for jupyter-latex-envs ... - \ done
  Stored in directory: /root/.cache/pip/wheels/65/c1/0f/b90b6d7032c79ee40181fdadb72b2ef9a186f02269d6c52f02
Successfully built jupyter-contrib-nbextensions jupyter-latex-envs
[I 12:28:22 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 12:28:22 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 12:28:22 InstallContribNbextensionsApp] Installing /usr/local/lib/python3.6/dist-packages/ju

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import keras
import keras.backend as K
import tensorflow as tf

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from PIL import Image
import seaborn as sns

sns.set(color_codes=True)

x = []
y = []

img_path = Path("/content/gdrive/My Drive/Colab Notebooks/dataset/images/")

for i in img_path.glob("*.jpg"):
  img = Image.open(i)
  a = keras.preprocessing.image.img_to_array(img)
  print(i, a.shape)
  x.append(a)
  ctr = float(i.stem.split("_")[1])
  y.append(ctr)

x = np.asarray(x, dtype=np.float32)
y = np.asarray(y, dtype=np.float32)

Using TensorFlow backend.


/content/gdrive/My Drive/Colab Notebooks/dataset/images/000139_0.317708213.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/000941_0.100676447.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/001544_0.761352729.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/000382_0.022318661.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/001820_0.130456668.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/000915_0.183638852.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/001788_0.007121805.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/001629_0.109031913.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/001335_3.174603175.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/000351_0.295339227.jpg (250, 300, 3)
/content/gdrive/My Drive/Colab Notebooks/dataset/images/001259_4.65726

In [0]:
from sklearn.model_selection import train_test_split

x = x.astype("float32")
x /= 255
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)